# Questions 
1. How many passengers flew into and out of Australia for each airline in each month of 2019?
2. In the last 6 months which port country was responsible for the most passengers coming into the country?
3. What is the 3 month rolling average of freight arriving to Australia for each airline throughout 2018?

In [1]:
import numpy as np
import pandas as pd

import requests

from datetime import datetime

from dateutil.relativedelta import relativedelta
from dateutil.rrule import rrule, MONTHLY

## Parameters

In [2]:
# id of dataset/resource: "international-airlines-airline-by-country-of-port-data"
resource_id="809c77d8-fd68-4a2c-806f-c63d64e69842"

# url to query via SQL
datastore_sql = "https://data.gov.au/data/api/3/action/datastore_search_sql?sql="

## Question 1
How many passengers flew into and out of Australia for each airline in each month of 2019?

In [3]:
# keep year as variable in case we want to test for other years
year = 2019 

In [4]:
# retrieve month, airline, passengers_in, passengers_out for year 2019
query = """
SELECT \"Month\", \"Month_num\", \"Airline\", \"Passengers_In\", \"Passengers_Out\"
FROM \"{}\"
WHERE \"Year\" = \'{}\';
""".format(resource_id, year)

print("Query to retrieve data for question 1:\n{}".format(query))

Query to retrieve data for question 1:

SELECT "Month", "Month_num", "Airline", "Passengers_In", "Passengers_Out"
FROM "809c77d8-fd68-4a2c-806f-c63d64e69842"
WHERE "Year" = '2019';



In [5]:
# submit datastore search request using above formatted query
response = requests.get(datastore_sql + query)

# check that GET request returned a successful response
assert response.json()['success'] is True, "GET request failed--query unsuccessful."

# keep result of response
records = response.json()['result']['records']
print("Data returned by query:\n{}".format(records))

Data returned by query:
[{'Month_num': '1', 'Passengers_In': '1690', 'Passengers_Out': '1411', 'Airline': 'Solomon Airlines', 'Month': 'Jan-19'}, {'Month_num': '4', 'Passengers_In': '3111', 'Passengers_Out': '2860', 'Airline': 'Qantas Airways', 'Month': 'Apr-19'}, {'Month_num': '5', 'Passengers_In': '0', 'Passengers_Out': '0', 'Airline': 'Polar Air Cargo', 'Month': 'May-19'}, {'Month_num': '12', 'Passengers_In': '2870', 'Passengers_Out': '3067', 'Airline': 'Air Mauritius', 'Month': 'Dec-19'}, {'Month_num': '11', 'Passengers_In': '59478', 'Passengers_Out': '55405', 'Airline': 'Qantas Airways', 'Month': 'Nov-19'}, {'Month_num': '3', 'Passengers_In': '1641', 'Passengers_Out': '1873', 'Airline': 'Air Mauritius', 'Month': 'Mar-19'}, {'Month_num': '4', 'Passengers_In': '1512', 'Passengers_Out': '1302', 'Airline': 'Solomon Airlines', 'Month': 'Apr-19'}, {'Month_num': '10', 'Passengers_In': '7833', 'Passengers_Out': '5678', 'Airline': 'Royal Brunei Airlines', 'Month': 'Oct-19'}, {'Month_num': 

In [6]:
# convert to pandas dataframe
df_1 = pd.DataFrame(records)

# replace any 0s or nones in Passengers_In 
df_1['Passengers_In'].fillna(0, inplace=True)
df_1['Passengers_Out'].fillna(0, inplace=True)

# make sure that Passengers_In and Passengers_Out are int dtypes
df_1[['Month_num', 'Passengers_In', 'Passengers_Out']] = df_1[['Month_num', 'Passengers_In', 'Passengers_Out']].astype('int64')

# groupby on airline, month and sum to get answer to Q1
df_1 = df_1.groupby(['Month', 'Month_num', 'Airline']).sum().sort_values(by=['Month_num'])
# get rid of Month_num now that we have sorted
df_1.index = df_1.index.droplevel('Month_num')

# print total number of passengers in and passengers out
print("Total number of passengers into Australia in 2019: {}".format(df_1.Passengers_In.sum()))
print("Total number of passengers out of Australia in 2019: {}".format(df_1.Passengers_Out.sum()))

# show df as well
df_1

Total number of passengers into Australia in 2019: 21350917
Total number of passengers out of Australia in 2019: 21157538


Passengers_In  Passengers_Out
Month  Airline                                                   
Jan-19 Cebu Pacific Air                     16158           13545
       Hainan Airlines                      13380           10477
       Garuda Indonesia                     36343           27185
       Fiji Airways                         26844           22497
       Federal Express Corporation              0               0
...                                           ...             ...
Dec-19 Garuda Indonesia                     22271           30714
       Hainan Airlines                       6257           10791
       Hawaiian Airlines                     9060           11076
       Japan Airlines                        9414           11635
       Air Canada                           16792           20523

[745 rows x 2 columns]

In [8]:
# export results to a csv for validation
df_1.reset_index().to_csv('question_1.csv', index=False, header=True)

## Question 2
In the last 6 months which port country was responsible for the most passengers coming into the country?<br>
(NOTE: dataset appears to be last refreshed in June 2021)

In [9]:
# fetch last 6 months from current date
# keep as variable in case we want to change it to another number of months
n_months = 6

In [10]:
end_period = datetime.now()
start_period = end_period - relativedelta(months=n_months)
print("Today's date is: {} and {} months ago from today was: {}.".format(end_period, n_months, start_period))

Today's date is: 2021-09-11 22:56:57.169905 and 6 months ago from today was: 2021-03-11 22:56:57.169905.


In [11]:
# retrieve port_country, passengers_in, as well as month_num and year for filtering df
# get filter on start_period.year, end_period.year so we only retrieve relevant years of data
query = """
SELECT \"Month_num\", \"Year\", \"Port_Country\", \"Passengers_In\"
FROM \"{}\"
WHERE \"Year\" IN ('{}', '{}');
""".format(resource_id, start_period.year, end_period.year)

print("Query to retrieve data for question 2:\n{}".format(query))

Query to retrieve data for question 2:

SELECT "Month_num", "Year", "Port_Country", "Passengers_In"
FROM "809c77d8-fd68-4a2c-806f-c63d64e69842"
WHERE "Year" IN ('2021', '2021');



In [12]:
# submit datastore search request using above formatted query
response = requests.get(datastore_sql + query)

# check that GET request returned a successful response
assert response.json()['success'] is True, "GET request failed--query unsuccessful."

# keep result of response
records = response.json()['result']['records']
print("Data returned by query:\n{}".format(records))

Data returned by query:
[{'Month_num': '1', 'Passengers_In': '1161', 'Port_Country': 'USA', 'Year': '2021'}, {'Month_num': '6', 'Passengers_In': '72', 'Port_Country': 'Nauru', 'Year': '2021'}, {'Month_num': '1', 'Passengers_In': '0', 'Port_Country': 'Hong Kong (SAR)', 'Year': '2021'}, {'Month_num': '1', 'Passengers_In': '0', 'Port_Country': 'USA', 'Year': '2021'}, {'Month_num': '1', 'Passengers_In': '0', 'Port_Country': 'China', 'Year': '2021'}, {'Month_num': '1', 'Passengers_In': '443', 'Port_Country': 'New Zealand', 'Year': '2021'}, {'Month_num': '5', 'Passengers_In': '305', 'Port_Country': 'Indonesia', 'Year': '2021'}, {'Month_num': '2', 'Passengers_In': '0', 'Port_Country': 'Solomon Islands', 'Year': '2021'}, {'Month_num': '1', 'Passengers_In': '89', 'Port_Country': 'Philippines', 'Year': '2021'}, {'Month_num': '4', 'Passengers_In': '0', 'Port_Country': 'Japan', 'Year': '2021'}, {'Month_num': '6', 'Passengers_In': '0', 'Port_Country': 'Vietnam', 'Year': '2021'}, {'Month_num': '2', 

In [13]:
# convert to pandas dataframe
df_2 = pd.DataFrame(records)

# replace any 0s or nones in Passengers_In 
df_2['Passengers_In'].fillna(0, inplace=True)

# make sure that Passengers_In and Passengers_Out are int dtypes
df_2[['Month_num', 'Year', 'Passengers_In']] = df_2[['Month_num', 'Year', 'Passengers_In']].astype('int64')

# sanity check: print min and max month_num, year
print("Sanity check pre-filter:\nMin month: {}\nMax month: {}".format(df_2.Month_num.min(), df_2.Month_num.max()))
print("Sanity check pre-filter:\nMin year: {}\nMax year: {}".format(df_2.Year.min(), df_2.Year.max()))

# create a temp column of "mm-yyyy" to use as a filter 
# alternatively could create a datetime column but this seems easiest
df_2['month_year'] = df_2.Month_num.astype('str') + '-' + df_2.Year.astype('str')

# generate list of month/year combinations between start_period (n_months ago) and end_period (now)
# for documentation see: https://dateutil.readthedocs.io/en/stable/rrule.html 
filt_dates = [(str(dt.month) + '-' + str(dt.year)) for dt in rrule(MONTHLY, dtstart=start_period, until=end_period)]

# now only keep month_year combinations that are within the allowable filt_dates range
df_2 = df_2.loc[df_2.month_year.isin(filt_dates)].copy()

# drop month_year column, unnecessary + reset index to clean up
df_2.drop(columns=['month_year'], inplace=True)
df_2.reset_index(inplace=True, drop=True)

# sanity check: print min and max month_num, year
print("\nSanity check post-filter:\nMin month: {}\nMax month: {}".format(df_2.Month_num.min(), df_2.Month_num.max()))
print("Sanity check post-filter:\nMin year: {}\nMax year: {}".format(df_2.Year.min(), df_2.Year.max()))

Sanity check pre-filter:
Min month: 1
Max month: 6
Sanity check pre-filter:
Min year: 2021
Max year: 2021

Sanity check post-filter:
Min month: 3
Max month: 6
Sanity check post-filter:
Min year: 2021
Max year: 2021


In [14]:
# get the total number of passengers in over the last n_month period, per port country
psng_by_port = df_2.drop(columns=['Month_num', 'Year']).groupby(['Port_Country']).Passengers_In.sum()

# export as .csv
psng_by_port.reset_index().to_csv('question_2.csv', index=False, header=True)

# print max country
max_psng = df_2.Passengers_In.max()
print("Country with most passengers in last {} months:".format(n_months))
print(df_2.loc[df_2.Passengers_In==max_psng][['Passengers_In', 'Port_Country']])
      
# also print df here in descending order
psng_by_port.reset_index().sort_values(by=['Passengers_In'], ascending=False)

Country with most passengers in last 6 months:
     Passengers_In Port_Country
213          53990  New Zealand


,Port_Country,Passengers_In
15,New Zealand,195671
19,Singapore,27285
18,Qatar,17946
25,United Arab Emirates,11760
24,USA,9930
10,Japan,3317
16,Papua New Guinea,3192
4,China,2908
7,Hong Kong (SAR),2428
9,Indonesia,1897


## Question 3
What is the 3 month rolling average of freight arriving to Australia for each airline throughout 2018?

In [15]:
# variables: year, rolling avg
# note: unlike q1 and q2, have not tested these with other values
year = 2018
monthly_roll_avg = 3

In [16]:
query = """
SELECT \"Month\", \"Month_num\", \"Year\", \"Airline\", \"Freight_In_(tonnes)\"
FROM \"{}\"
WHERE \"Year\" = \'{}\';
""".format(resource_id, year)

print("Query to retrieve data for question 3:\n{}".format(query))

Query to retrieve data for question 3:

SELECT "Month", "Month_num", "Year", "Airline", "Freight_In_(tonnes)"
FROM "809c77d8-fd68-4a2c-806f-c63d64e69842"
WHERE "Year" = '2018';



In [17]:
# submit datastore search request using above formatted query
response = requests.get(datastore_sql + query)

# check that GET request returned a successful response
assert response.json()['success'] is True, "GET request failed--query unsuccessful."

# keep result of response
records = response.json()['result']['records']
print("Data returned by query:\n{}".format(records))

Data returned by query:
[{'Freight_In_(tonnes)': '0', 'Month_num': '3', 'Year': '2018', 'Airline': 'Virgin Australia', 'Month': 'Mar-18'}, {'Freight_In_(tonnes)': '38.315', 'Month_num': '3', 'Year': '2018', 'Airline': 'Emirates', 'Month': 'Mar-18'}, {'Freight_In_(tonnes)': '429.366', 'Month_num': '2', 'Year': '2018', 'Airline': 'Virgin Australia', 'Month': 'Feb-18'}, {'Freight_In_(tonnes)': '44.154', 'Month_num': '7', 'Year': '2018', 'Airline': 'LATAM Airlines', 'Month': 'Jul-18'}, {'Freight_In_(tonnes)': '0', 'Month_num': '5', 'Year': '2018', 'Airline': 'Samoa Airways', 'Month': 'May-18'}, {'Freight_In_(tonnes)': '3065.944', 'Month_num': '1', 'Year': '2018', 'Airline': 'Air New Zealand', 'Month': 'Jan-18'}, {'Freight_In_(tonnes)': '1944.932', 'Month_num': '8', 'Year': '2018', 'Airline': 'Malaysia Airlines', 'Month': 'Aug-18'}, {'Freight_In_(tonnes)': '1738.798', 'Month_num': '10', 'Year': '2018', 'Airline': 'Qantas Airways', 'Month': 'Oct-18'}, {'Freight_In_(tonnes)': '0', 'Month_num'

In [18]:
# convert to pandas dataframe
df_3 = pd.DataFrame(records)

# create a datetime column so we can use pandas native rolling avg
df_3['date'] = df_3.Month_num.astype('str') + '/' + df_3.Year.astype('str')
df_3['date'] = pd.to_datetime(df_3['date'])

# make sure we have correct dtypes and fill nans
df_3['Freight_In_(tonnes)'].fillna(0, inplace=True)

# make sure that Passengers_In and Passengers_Out are int dtypes
df_3[['Month_num', 'Year']] = df_3[['Month_num', 'Year']].astype('int64')
df_3['Freight_In_(tonnes)'] = df_3['Freight_In_(tonnes)'].astype('float')

# drop Month_num, Year columns
df_3.drop(columns=['Month_num', 'Year', 'Month'], inplace=True)

# group by Airline, date and get sum in case of duplicates
df_3 = df_3.groupby(['Airline', 'date']).sum().reset_index()

# order by date
df_3.sort_values(by=['Airline', 'date'], inplace=True)

# check current df
df_3

,Airline,date,Freight_In_(tonnes)
0,Air Caledonie International,2018-01-01,2.942
1,Air Caledonie International,2018-02-01,4.546
2,Air Caledonie International,2018-03-01,4.184
3,Air Caledonie International,2018-04-01,8.042
4,Air Caledonie International,2018-05-01,2.332
...,...,...,...
732,Xiamen Airlines,2018-08-01,462.732
733,Xiamen Airlines,2018-09-01,452.296
734,Xiamen Airlines,2018-10-01,458.562
735,Xiamen Airlines,2018-11-01,495.361


In [19]:
# now get 3 month rolling average per airline. use panda's rolling with transform
# see documentation: https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.transform.html#pandas-core-groupby-dataframegroupby-transform
# see documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html#pandas-dataframe-rolling
df_3['moving_avg'] = df_3.groupby(['Airline'])['Freight_In_(tonnes)'].transform(lambda x: x.rolling(3, 1).mean())
df_3

,Airline,date,Freight_In_(tonnes),moving_avg
0,Air Caledonie International,2018-01-01,2.942,2.942000
1,Air Caledonie International,2018-02-01,4.546,3.744000
2,Air Caledonie International,2018-03-01,4.184,3.890667
3,Air Caledonie International,2018-04-01,8.042,5.590667
4,Air Caledonie International,2018-05-01,2.332,4.852667
...,...,...,...,...
732,Xiamen Airlines,2018-08-01,462.732,485.942000
733,Xiamen Airlines,2018-09-01,452.296,493.626000
734,Xiamen Airlines,2018-10-01,458.562,457.863333
735,Xiamen Airlines,2018-11-01,495.361,468.739667


In [20]:
# export to csv
df_3.to_csv('question_3.csv', index=False, header=True)